In [ ]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split

import optuna

import lightgbm as lgbm

import sklearn.datasets
import sklearn.metrics

DEFAULT_RANDOM_SEED = 2021
def seedBasic(seed=DEFAULT_RANDOM_SEED):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

# global variabels

In [ ]:
TRAIN_PATH = "../input/tabular-playground-series-dec-2021/train.csv"
TEST_PATH = "../input/tabular-playground-series-dec-2021/test.csv"
SAMPLE_SUBMISSION_PATH = "../input/tabular-playground-series-dec-2021/sample_submission.csv"
SUBMISSION_PATH = "submission.csv"

ID = "Id"
TARGET = "Cover_Type"

N_TRIALS = 5
DIRECTIONS = "maximize"
TEST_SIZE = 0.25

# load data 

In [ ]:
train = pd.read_csv(TRAIN_PATH)
test = pd.read_csv(TEST_PATH)

# quick preprocess

# split data (input data and target data)

In [ ]:
y = train[TARGET]
X = train.drop([TARGET],axis=1)
X_test = test

# split data (train data and validataion data)

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=TEST_SIZE)

# make study and search best parameters

In [ ]:
def objective(trial):
   
    # 1.make train data for optuna 
    train_dataset = lgbm.Dataset(X_train, label=y_train)
    
    # 2.train
    param = {
        "verbosity": -1,
        "boosting_type": "gbdt",
        "lambda_l1": trial.suggest_float("lambda_l1", 1e-8, 10.0, log=True),
        "lambda_l2": trial.suggest_float("lambda_l2", 1e-8, 10.0, log=True),
        "num_leaves": trial.suggest_int("num_leaves", 2, 256),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.4, 1.0),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.4, 1.0),
        "bagging_freq": trial.suggest_int("bagging_freq", 1, 7),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
    }
    optunaModel = lgbm.train(param, train_dataset)
    
    # 3.predict
    preds = optunaModel.predict(X_val)
    
    # 4.evaluate
    pred_labels = np.rint(preds)
    accuracy = sklearn.metrics.accuracy_score(y_val, pred_labels)
    
    return accuracy

study = optuna.create_study(direction=DIRECTIONS)
study.optimize(objective, n_trials=N_TRIALS)

# study result 

In [ ]:
study.best_trial

In [ ]:
study.best_trial.value

# best params

In [ ]:
study.best_trial.params

# build model using optuna best model

In [ ]:
model = lgbm.LGBMClassifier(**study.best_trial.params)

In [ ]:
model.fit(X_train,y_train)

# predict test data target 

In [ ]:
pred_test = model.predict(X_test)

# submission 

In [ ]:
sub = pd.read_csv(SAMPLE_SUBMISSION_PATH)
sub[TARGET] = pred_test
sub.to_csv(SUBMISSION_PATH,index=False)
sub.head()